Q1: How many convolutional layers are in the ResNet-18 model ?

Ans - 17

Q2: What is the input dimension of the last linear layer ?

Ans - 512

Q3: How many trainable parameters and how many gradients in the ResNet-18 model that you build when using SGD

In [23]:
from resnet_test import *
import torch
from torch.optim import SGD, Adagrad, Adadelta, Adam
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.datasets import CIFAR10


In [2]:
net = ResNet18()

In [11]:
def get_params(net):
    return sum(p.numel() for p in net.parameters() if p.requires_grad)

def get_gradients(net):

In [12]:
get_params(net)

11173962

In [25]:
transform_train = transforms.Compose([
	transforms.RandomCrop(32, padding=4),
	transforms.RandomHorizontalFlip(),
	transforms.ToTensor(),
	transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)


Files already downloaded and verified


In [ ]:
iter_loader = iter(trainloader)
image, labels = next(iter_loader)
out = net(image)
loss = F.cross_entropy(out, labels)

optimizer.zero_grad()